# ML Models for Trading - Chapters 08-12

This notebook covers the machine learning cluster from the Puffin project, spanning:

- **Chapter 08**: Linear models (OLS, Ridge, Lasso, Logistic Regression, Fama-French)
- **Chapter 09**: Time series models (stationarity, ARIMA, VAR, GARCH, cointegration)
- **Chapter 10**: Bayesian ML (covered separately)
- **Chapter 11**: Tree ensembles (Random Forest, XGBoost, LightGBM, SHAP)
- **Chapter 12**: Unsupervised learning (PCA, clustering, data-driven risk factors)

Each section imports from the `puffin` package and demonstrates the model APIs with synthetic or downloaded data.

## 1. Linear Models

Linear regression models form the baseline for return prediction and factor analysis.
We demonstrate OLS, Ridge (L2), Lasso (L1), and a direction classifier (logistic regression).

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

from puffin.models import OLSModel, RidgeModel, LassoModel, DirectionClassifier

# Generate synthetic return prediction data
np.random.seed(42)
n = 500
dates = pd.date_range('2020-01-01', periods=n, freq='B')

features = pd.DataFrame({
    'momentum_5d': np.random.randn(n) * 0.02,
    'momentum_20d': np.random.randn(n) * 0.03,
    'volatility_20d': np.abs(np.random.randn(n)) * 0.01,
    'volume_ratio': 1 + np.random.randn(n) * 0.3,
    'rsi_14': 50 + np.random.randn(n) * 15,
}, index=dates)

# Target: next-day returns with some signal from features
returns = (0.3 * features['momentum_5d'] + 0.2 * features['momentum_20d']
           - 0.1 * features['volatility_20d'] + np.random.randn(n) * 0.01)
returns.name = 'forward_return'

# Train/test split (80/20)
split = int(n * 0.8)
X_train, X_test = features.iloc[:split], features.iloc[split:]
y_train, y_test = returns.iloc[:split], returns.iloc[split:]

# --- OLS ---
ols = OLSModel().fit(X_train, y_train)
ols_summary = ols.summary()
print("=== OLS Model ===")
print(f"R-squared: {ols_summary['r_squared']:.4f}")
print(f"Significant features (p < 0.05):")
for name, pval in ols.p_values.items():
    if pval < 0.05:
        print(f"  {name}: coef={ols.coefficients[name]:.6f}, p={pval:.4f}")

# --- Ridge ---
ridge = RidgeModel().fit(X_train, y_train)
print(f"\n=== Ridge Model (alpha={ridge.alpha:.4f}) ===")
print("Feature importance:")
print(ridge.feature_importance().head())

# --- Lasso ---
lasso = LassoModel().fit(X_train, y_train)
print(f"\n=== Lasso Model (alpha={lasso.alpha:.6f}) ===")
print(f"Selected features: {lasso.selected_features}")
print(f"Coefficients:\n{lasso.coefficients}")

# --- Direction Classifier ---
direction = (y_train > 0).astype(int)
clf = DirectionClassifier().fit(X_train, direction)
test_direction = (y_test > 0).astype(int)
preds = clf.predict(X_test)
accuracy = (preds == test_direction.values).mean()
print(f"\n=== Direction Classifier ===")
print(f"Test accuracy: {accuracy:.2%}")
print(f"Feature importance:\n{clf.feature_importance()}")

## 2. Fama-French Factor Models

The Fama-French model explains asset returns using common risk factors: market, size (SMB),
value (HML), profitability (RMW), and investment (CMA). We fit CAPM, 3-factor, and 5-factor models.

In [ ]:
from puffin.models import FamaFrenchModel

ff = FamaFrenchModel()

# Generate synthetic asset returns for demonstration
np.random.seed(42)
n_days = 504  # ~2 years of trading days
dates = pd.date_range('2022-01-01', periods=n_days, freq='B')

# Simulate returns with known factor exposures
factors = ff.fetch_factors('2022-01-01', '2024-01-01')
factors = factors.iloc[:n_days]  # Align length
factors.index = dates

# Asset with beta=1.2, positive SMB, negative HML
asset_returns = pd.Series(
    0.0002 + 1.2 * factors['Mkt-RF'] + 0.5 * factors['SMB']
    - 0.3 * factors['HML'] + factors['RF'] + np.random.randn(n_days) * 0.005,
    index=dates
)

# CAPM
capm = ff.fit_capm(asset_returns, start='2022-01-01', end='2024-01-01')
print("=== CAPM ===")
print(f"Alpha: {capm['alpha']:.6f} (p={capm['alpha_pvalue']:.4f})")
print(f"Beta:  {capm['beta']:.4f} (p={capm['beta_pvalue']:.4f})")
print(f"R-squared: {capm['r_squared']:.4f}")

# 3-Factor Model
ff3 = ff.fit_three_factor(asset_returns, start='2022-01-01', end='2024-01-01')
print(f"\n=== Fama-French 3-Factor ===")
print(f"Alpha: {ff3['alpha']:.6f}")
for factor, beta in ff3['betas'].items():
    pval = ff3['pvalues'][factor]
    print(f"  {factor}: beta={beta:.4f} (p={pval:.4f})")
print(f"R-squared: {ff3['r_squared']:.4f}")

# 5-Factor Model
ff5 = ff.fit_five_factor(asset_returns, start='2022-01-01', end='2024-01-01')
print(f"\n=== Fama-French 5-Factor ===")
print(f"Alpha: {ff5['alpha']:.6f}")
for factor, beta in ff5['betas'].items():
    pval = ff5['pvalues'][factor]
    print(f"  {factor}: beta={beta:.4f} (p={pval:.4f})")
print(f"R-squared: {ff5['r_squared']:.4f}")

## 3. Time Series Diagnostics

Before fitting time series models, we must check for stationarity and decompose the series
into trend, seasonal, and residual components.

In [ ]:
from puffin.models import test_stationarity, decompose_series

# Generate a non-stationary price series and its stationary returns
np.random.seed(42)
n = 600
dates = pd.date_range('2020-01-01', periods=n, freq='B')
returns_series = pd.Series(np.random.randn(n) * 0.01 + 0.0003, index=dates)
prices = (1 + returns_series).cumprod() * 100

# Test stationarity on prices vs returns
print("=== Prices (expected: non-stationary) ===")
price_result = test_stationarity(prices)
print(f"ADF statistic: {price_result['test_statistic']:.4f}")
print(f"p-value: {price_result['p_value']:.4f}")
print(f"Is stationary: {price_result['is_stationary']}")

print(f"\n=== Returns (expected: stationary) ===")
ret_result = test_stationarity(returns_series)
print(f"ADF statistic: {ret_result['test_statistic']:.4f}")
print(f"p-value: {ret_result['p_value']:.4f}")
print(f"Is stationary: {ret_result['is_stationary']}")

# Decompose the price series
components = decompose_series(prices, period=63)  # Quarterly seasonality
fig, axes = plt.subplots(4, 1, figsize=(14, 10), sharex=True)
for ax, (name, data) in zip(axes, components.items()):
    ax.plot(data)
    ax.set_ylabel(name.capitalize())
axes[0].set_title('Time Series Decomposition')
plt.tight_layout()
plt.show()

## 4. ARIMA Models

ARIMA models combine autoregressive (AR), differencing (I), and moving average (MA) components.
We fit a manual ARIMA and use `auto_arima` for automatic order selection.

In [ ]:
from puffin.models import ARIMAModel, auto_arima

# Use the returns series from above
train_returns = returns_series.iloc[:480]
test_returns = returns_series.iloc[480:]

# Manual ARIMA(1,0,1) fit
model = ARIMAModel(order=(1, 0, 1))
model.fit(train_returns)
print(f"=== ARIMA(1,0,1) ===")
print(f"AIC: {model.aic_:.2f}")
print(f"BIC: {model.bic_:.2f}")

# Forecast with confidence intervals
forecast_df = model.forecast(train_returns, horizon=20, confidence=0.95)
print(f"\n20-step forecast (first 5 rows):")
print(forecast_df.head())

# Auto-ARIMA: search for optimal order
auto_model = auto_arima(train_returns, max_p=3, max_d=1, max_q=3)
print(f"\n=== Auto ARIMA ===")
print(f"Selected order: {auto_model.order_}")
print(f"AIC: {auto_model.aic_:.2f}")

# Plot forecast
forecast_df = auto_model.forecast(train_returns, horizon=30)
fig, ax = plt.subplots(figsize=(14, 5))
train_returns.iloc[-60:].plot(ax=ax, label='Historical')
forecast_df['forecast'].plot(ax=ax, label='Forecast', color='red')
ax.fill_between(forecast_df.index, forecast_df['lower'], forecast_df['upper'],
                alpha=0.2, color='red', label='95% CI')
ax.legend()
ax.set_title(f'ARIMA{auto_model.order_} Forecast')
plt.tight_layout()
plt.show()

## 5. VAR & GARCH Models

**VAR** captures linear interdependencies among multiple time series.
**GARCH** models time-varying volatility and volatility clustering in financial returns.

In [ ]:
from puffin.models import VARModel, GARCHModel

# --- VAR Model ---
# Simulate two correlated return series
np.random.seed(42)
n = 500
dates = pd.date_range('2020-01-01', periods=n, freq='B')
eps1 = np.random.randn(n) * 0.01
eps2 = np.random.randn(n) * 0.01
r1 = np.zeros(n)
r2 = np.zeros(n)
for t in range(1, n):
    r1[t] = 0.3 * r1[t-1] + 0.1 * r2[t-1] + eps1[t]
    r2[t] = 0.05 * r1[t-1] + 0.4 * r2[t-1] + eps2[t]

var_data = pd.DataFrame({'stock_A': r1, 'stock_B': r2}, index=dates)

var_model = VARModel()
var_model.fit(var_data, max_lags=5)
print(f"=== VAR Model (lags={var_model.lags_}) ===")

# Forecast
var_forecast = var_model.predict(steps=5)
print(f"5-step forecast:\n{var_forecast}")

# Impulse response
irf = var_model.impulse_response(periods=20)
fig, axes = plt.subplots(1, 2, figsize=(14, 4))
axes[0].plot(irf[:, 0, 0], label='A -> A')
axes[0].plot(irf[:, 1, 0], label='A -> B')
axes[0].set_title('Impulse Response: Shock to Stock A')
axes[0].legend()
axes[1].plot(irf[:, 0, 1], label='B -> A')
axes[1].plot(irf[:, 1, 1], label='B -> B')
axes[1].set_title('Impulse Response: Shock to Stock B')
axes[1].legend()
plt.tight_layout()
plt.show()

# --- GARCH Model ---
np.random.seed(42)
n = 1000
dates = pd.date_range('2018-01-01', periods=n, freq='B')
returns_garch = pd.Series(np.random.randn(n) * 0.015, index=dates)
# Add volatility clustering
for t in range(1, n):
    vol = 0.005 + 0.7 * abs(returns_garch.iloc[t-1])
    returns_garch.iloc[t] = np.random.randn() * vol

garch = GARCHModel(p=1, q=1)
garch.fit(returns_garch * 100)  # Scale to percentage
print(f"\n=== GARCH(1,1) ===")
params = garch.get_params()
print(f"AIC: {params['aic']:.2f}")
print(f"Parameters:\n{params['params']}")

# Plot conditional volatility
fig, ax = plt.subplots(figsize=(14, 4))
ax.plot(garch.conditional_volatility, label='Conditional Volatility')
ax.set_title('GARCH(1,1) Conditional Volatility')
ax.legend()
plt.tight_layout()
plt.show()

## 6. Cointegration & Pairs Trading

Cointegration identifies long-run equilibrium relationships between price series.
We use it to build a pairs trading strategy that profits from mean-reverting spreads.

In [ ]:
from puffin.models import find_cointegrated_pairs, PairsTradingStrategy

# Simulate cointegrated pairs
np.random.seed(42)
n = 500
dates = pd.date_range('2020-01-01', periods=n, freq='B')

# Cointegrated pair: stock_A and stock_B share a common stochastic trend
common_trend = np.random.randn(n).cumsum()
stock_A = 100 + common_trend + np.random.randn(n) * 0.5
stock_B = 50 + 0.5 * common_trend + np.random.randn(n) * 0.3

# Non-cointegrated stock
stock_C = 80 + np.random.randn(n).cumsum()

prices = pd.DataFrame({
    'stock_A': stock_A,
    'stock_B': stock_B,
    'stock_C': stock_C,
}, index=dates)

# Find cointegrated pairs
pairs = find_cointegrated_pairs(prices, significance=0.05)
print("=== Cointegrated Pairs ===")
for t1, t2, pval, hr in pairs:
    print(f"  {t1} - {t2}: p-value={pval:.4f}, hedge_ratio={hr:.4f}")

# Pairs trading strategy
strategy = PairsTradingStrategy(entry_z=2.0, exit_z=0.5, lookback=20)

if pairs:
    best_pair = (pairs[0][0], pairs[0][1])
    hedge_ratio = pairs[0][3]

    # Compute spread and signals
    spread = strategy.compute_spread(best_pair, prices, hedge_ratio=hedge_ratio)
    signals = strategy.generate_signals(spread)

    # Backtest
    results = strategy.backtest_pair(best_pair, prices, hedge_ratio=hedge_ratio)
    print(f"\n=== Pairs Trading Backtest: {best_pair[0]}-{best_pair[1]} ===")
    print(f"Sharpe Ratio: {results['sharpe_ratio']:.2f}")
    print(f"Total Return: {results['total_return']:.2%}")
    print(f"Max Drawdown: {results['max_drawdown']:.2%}")
    print(f"Win Rate: {results['win_rate']:.2%}")
    print(f"Number of Trades: {results['num_trades']}")

    # Plot spread and signals
    fig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True)
    spread.plot(ax=axes[0], label='Spread')
    mean = spread.rolling(20).mean()
    std = spread.rolling(20).std()
    axes[0].plot(mean, 'r--', label='Mean')
    axes[0].fill_between(spread.index, mean - 2*std, mean + 2*std, alpha=0.1, color='red')
    axes[0].legend()
    axes[0].set_title('Spread with Bollinger Bands')

    results['cumulative_returns'].plot(ax=axes[1], label='Strategy')
    axes[1].set_title('Cumulative Returns')
    axes[1].legend()
    plt.tight_layout()
    plt.show()

## 7. Tree Ensemble Models

Tree ensembles aggregate many decision trees for robust predictions.
We compare Random Forest, XGBoost, and LightGBM on a directional trading task.

In [ ]:
from puffin.ensembles import RandomForestTrader, XGBoostTrader, LightGBMTrader

# Create feature set for direction prediction
np.random.seed(42)
n = 800
dates = pd.date_range('2019-01-01', periods=n, freq='B')

X = pd.DataFrame({
    'mom_5': np.random.randn(n) * 0.02,
    'mom_10': np.random.randn(n) * 0.025,
    'mom_20': np.random.randn(n) * 0.03,
    'vol_10': np.abs(np.random.randn(n)) * 0.01,
    'vol_20': np.abs(np.random.randn(n)) * 0.012,
    'rsi': 50 + np.random.randn(n) * 15,
    'macd': np.random.randn(n) * 0.005,
    'bb_width': np.abs(np.random.randn(n)) * 0.02,
}, index=dates)

# Generate labels with some predictability
signal = 0.3 * X['mom_5'] + 0.2 * X['macd'] - 0.1 * X['vol_20']
y = (signal + np.random.randn(n) * 0.005 > 0).astype(int)
y = pd.Series(y, index=dates, name='direction')

split = int(n * 0.75)
X_train, X_test = X.iloc[:split], X.iloc[split:]
y_train, y_test = y.iloc[:split], y.iloc[split:]

results = {}

# Random Forest
rf = RandomForestTrader(task='classification')
rf.fit(X_train, y_train, n_estimators=200, max_depth=8)
rf_preds = rf.predict(X_test)
rf_acc = np.nanmean(rf_preds == y_test.values)
results['RandomForest'] = rf_acc
print(f"Random Forest accuracy: {rf_acc:.2%}")

# XGBoost
if XGBoostTrader is not None:
    xgb_model = XGBoostTrader(task='classification')
    xgb_model.fit(X_train, y_train)
    xgb_preds = xgb_model.predict(X_test)
    xgb_acc = np.nanmean(xgb_preds == y_test.values)
    results['XGBoost'] = xgb_acc
    print(f"XGBoost accuracy: {xgb_acc:.2%}")
else:
    print("XGBoost not installed, skipping.")

# LightGBM
if LightGBMTrader is not None:
    lgb_model = LightGBMTrader(task='classification')
    lgb_model.fit(X_train, y_train)
    lgb_preds = lgb_model.predict(X_test)
    lgb_acc = np.nanmean(lgb_preds == y_test.values)
    results['LightGBM'] = lgb_acc
    print(f"LightGBM accuracy: {lgb_acc:.2%}")
else:
    print("LightGBM not installed, skipping.")

# Compare feature importances
print("\n=== Feature Importance (Random Forest) ===")
print(rf.feature_importance())

# Cross-validation
cv_results = rf.cross_validate(X, y, n_estimators=100, max_depth=8)
print(f"\nTime-series CV: mean={cv_results['mean_score']:.4f}, std={cv_results['std_score']:.4f}")

## 8. Model Interpretation with SHAP

SHAP (SHapley Additive exPlanations) values explain individual predictions by attributing
each feature's contribution. This is critical for understanding why a model generates a
particular trading signal.

In [ ]:
from puffin.ensembles import ModelInterpreter

if ModelInterpreter is not None:
    interpreter = ModelInterpreter()

    # Compute SHAP values for the Random Forest model
    shap_vals = interpreter.shap_values(rf.model, X_test)
    print(f"SHAP values shape: {shap_vals.values.shape}")
    print(f"Mean |SHAP| per feature:")
    mean_shap = pd.Series(
        np.abs(shap_vals.values).mean(axis=0),
        index=X_test.columns
    ).sort_values(ascending=False)
    print(mean_shap)

    # Summary plot
    fig = interpreter.plot_summary(rf.model, X_test, plot_type='bar', max_display=8)
    plt.show()

    # Waterfall plot for a single prediction
    fig = interpreter.plot_waterfall(rf.model, X_test, index=0)
    plt.show()

    # Compare models if XGBoost is available
    models_dict = {'RandomForest': rf.model}
    if XGBoostTrader is not None:
        models_dict['XGBoost'] = xgb_model.model
    if LightGBMTrader is not None:
        models_dict['LightGBM'] = lgb_model.model

    if len(models_dict) > 1:
        comparison = interpreter.feature_importance_comparison(models_dict, X_test, method='native')
        print("\n=== Feature Importance Comparison ===")
        print(comparison.head(8))
else:
    print("SHAP not installed. Install with: pip install shap")

## 9. Unsupervised Learning: PCA & Clustering

PCA extracts eigenportfolios (statistical risk factors) from the cross-section of returns.
Clustering groups assets with similar behavior, useful for sector rotation and diversification.

In [ ]:
from puffin.unsupervised import MarketPCA, cluster_assets

# Simulate a universe of 20 assets with sector structure
np.random.seed(42)
n_days = 504
n_assets = 20
dates = pd.date_range('2022-01-01', periods=n_days, freq='B')

# 3 hidden factors (market, sector1, sector2)
market_factor = np.random.randn(n_days) * 0.01
sector1_factor = np.random.randn(n_days) * 0.008
sector2_factor = np.random.randn(n_days) * 0.006

returns_matrix = np.zeros((n_days, n_assets))
for i in range(n_assets):
    beta_mkt = 0.8 + np.random.rand() * 0.4
    beta_s1 = (0.5 if i < 7 else -0.3) * np.random.rand()
    beta_s2 = (0.5 if 7 <= i < 14 else -0.2) * np.random.rand()
    returns_matrix[:, i] = (
        beta_mkt * market_factor + beta_s1 * sector1_factor
        + beta_s2 * sector2_factor + np.random.randn(n_days) * 0.005
    )

tickers = [f'STOCK_{i:02d}' for i in range(n_assets)]
returns_df = pd.DataFrame(returns_matrix, index=dates, columns=tickers)

# --- PCA ---
pca = MarketPCA()
pca.fit(returns_df)
print("=== PCA Analysis ===")
print(f"Components for 95% variance: {pca.n_components_95}")
print(f"Top 5 explained variance ratios: {pca.explained_variance_ratio[:5].round(4)}")

# Plot explained variance
var_data = pca.explained_variance_plot()
fig, ax = plt.subplots(figsize=(10, 5))
ax.bar(var_data['component'], var_data['variance_explained'], alpha=0.6, label='Individual')
ax.plot(var_data['component'], var_data['cumulative_variance'], 'r-o', label='Cumulative')
ax.axhline(y=0.95, color='g', linestyle='--', label='95% threshold')
ax.set_xlabel('Component')
ax.set_ylabel('Variance Explained')
ax.set_title('PCA Scree Plot')
ax.legend()
plt.tight_layout()
plt.show()

# Eigenportfolios
eigenportfolios = pca.eigenportfolios(returns_df, n=3)
print("\nTop 3 Eigenportfolio Weights:")
print(eigenportfolios.round(3))

# --- Clustering ---
labels = cluster_assets(returns_df, n_clusters=3, method='kmeans')
print(f"\n=== Clustering ===")
for cluster_id in np.unique(labels):
    assets_in_cluster = [tickers[i] for i in range(n_assets) if labels[i] == cluster_id]
    print(f"Cluster {cluster_id}: {assets_in_cluster}")

## 10. Data-Driven Risk Factors

We extract statistical risk factors from the cross-section of returns using PCA,
then compute factor exposures (loadings) and attribute asset returns to these factors.

In [ ]:
from puffin.unsupervised import extract_risk_factors, factor_attribution
from puffin.unsupervised.risk_factors import factor_exposures, factor_variance_decomposition

# Extract 5 risk factors from the returns universe
factors = extract_risk_factors(returns_df, n_factors=5)
print("=== Extracted Risk Factors ===")
print(f"Shape: {factors.shape}")
print(f"Factor correlations:\n{factors.corr().round(3)}")

# Factor exposures (loadings)
loadings = factor_exposures(returns_df, factors)
print(f"\nFactor Loadings (first 5 assets):")
print(loadings.head().round(4))

# Factor attribution
attribution = factor_attribution(returns_df, factors, loadings)
print(f"\nFactor-attributed returns shape: {attribution.shape}")

# Variance decomposition
var_decomp = factor_variance_decomposition(returns_df, n_factors=5)
print(f"\n=== Variance Decomposition (first 5 assets) ===")
print(var_decomp[['pct_factor', 'pct_specific']].head().round(1))

# Plot variance decomposition
fig, ax = plt.subplots(figsize=(14, 5))
var_decomp[['pct_factor', 'pct_specific']].plot(
    kind='bar', stacked=True, ax=ax, color=['steelblue', 'lightcoral']
)
ax.set_ylabel('Percentage of Total Variance')
ax.set_title('Factor vs Specific Variance Decomposition')
ax.set_xticklabels(var_decomp.index, rotation=45, ha='right')
ax.legend(['Factor Risk', 'Specific Risk'])
plt.tight_layout()
plt.show()

## Exercises

1. **Linear Models**: Replace synthetic data with real returns from `yfinance`. Compare OLS, Ridge, and Lasso R-squared on out-of-sample data. Which regularization helps most?
2. **ARIMA**: Use `auto_arima` on VIX data. Does the selected order change across different time windows?
3. **Pairs Trading**: Download prices for sector ETFs (XLF, XLK, XLE, etc.) and run `find_cointegrated_pairs`. Backtest the top pair.
4. **Tree Ensembles**: Tune XGBoost hyperparameters with `tune_hyperparameters()`. Does the tuned model beat the default on walk-forward validation?
5. **PCA**: Apply `MarketPCA` to S&P 500 constituents. How many components explain 90% of variance? Do eigenportfolios correspond to known sectors?